In [ ]:
CREATE OR REPLACE NETWORK RULE EN_NETWORK_RULE
MODE = EGRESS
TYPE = HOST_PORT
VALUE_LIST = ('US.ENGAGINGNETWORKS.APP');

In [ ]:
CREATE OR REPLACE SECRET EN_API_TOKEN_DB
TYPE = GENERIC_STRING

--SECRET_STRING='a8fda2f8-aff5-44e7-be10-6234309f3ce0' --Snowflake_EN_API user
SECRET_STRING='63f77a9d-266b-4063-ac29-d548fa9bfe57';   --Daina
--SECRET_STRING ='8a911743-eff6-40fd-b4ba-848a17f4ca9e';    --Jonathan

In [ ]:
CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION EN_INTEGRATION
ALLOWED_NETWORK_RULES = (EN_NETWORK_RULE)
ALLOWED_AUTHENTICATION_SECRETS = (EN_API_TOKEN_DB)
ENABLED=TRUE;

In [ ]:
CREATE OR REPLACE ROLE EN_DEVELOPER;
GRANT USAGE ON INTEGRATION EN_INTEGRATION TO ROLE EN_DEVELOPER;
GRANT READ ON SECRET EN_API_TOKEN_DB TO ROLE EN_DEVELOPER;

In [ ]:
import snowflake.snowpark as snowpark
import pandas as pd
     

# The Snowpark session is automatically managed within Snowflake
session = snowpark.Session.builder.getOrCreate()
     

# Load data from the Snowflake stage into a DataFrame
df = session.read.options({"field_delimiter": ",", "skip_header": 1}).csv("@TEST/P2P_EN_page_interactions_v2.csv")
df_pd = df.to_pandas()
df_pd.columns = ['Email Address', 'CLASSY_TEST', 'UNI_OPTIN']
df_pd

In [ ]:
CREATE OR REPLACE FUNCTION PROCESS_EN_PAGE()
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.11
HANDLER = 'process_en_page'
EXTERNAL_ACCESS_INTEGRATIONS = (EN_INTEGRATION)
PACKAGES = ('requests')
SECRETS = ('cred' = EN_API_TOKEN_DB)
IMPORTS=('@TEST/P2P_EN_page_interactions_v2.csv')
AS
$$
import requests
import _snowflake
import sys
import os

# File to upload
with open(os.path.join(sys._xoptions["snowflake_import_directory"], 'P2P_EN_page_interactions_v2.csv'), "r", encoding='utf-8') as f:
    file = f.read()

def process_en_page():
        
    # URL for the authentication endpoint
    url = "https://us.engagingnetworks.app/ea-dataservice/import.service"
        
    # Get the API token from Snowflake Secrets
    api_token = _snowflake.get_generic_secret_string('cred')  
    
    # Create a payload for the request
    data = {
        'token': api_token,
        'name': 'Test import from Snowflake',
        'formatName': 'classy_API_pp_v2'
    }
        
    
    # Send the request
    response = requests.post(url, data=data, files={'file': ('P2P_EN_page_interactions_v2.csv', file)})


    # Print response
    return response.text  # Return the response content as a string

$$;

In [ ]:
SELECT PROCESS_EN_PAGE()